# Data collection daily files
Gather public SPP Weis data from https://marketplace.spp.org/groups/operational-data-weis

In [0]:
%load_ext autoreload
%autoreload 2

In [0]:
import os
import sys
import pandas as pd

# logging
import logging

# define log
logging.basicConfig(level=logging.INFO)

# Set the logging level for the 'py4j' logger to ERROR or WARNING
logging.getLogger("py4j").setLevel(logging.ERROR) 

log = logging.getLogger(__name__)

In [0]:
# set env vars
if dbutils:
    os.environ['AWS_ACCESS_KEY_ID'] = dbutils.secrets.get(scope = "aws", key = "AWS_ACCESS_KEY_ID")
    os.environ['AWS_SECRET_ACCESS_KEY'] = dbutils.secrets.get(scope = "aws", key = "AWS_SECRET_ACCESS_KEY")

from dotenv import load_dotenv
load_dotenv()

In [0]:
os.chdir('../..')
sys.path.append('./src')

In [0]:
import src.data_collection as dc

## LMP settlement location prices

![_](../../imgs/lmp_settlement_location.PNG)

## LMP upsert

### LMP daily file

In [0]:
# getting ending time for data gathering
end_ts = (
    pd.Timestamp.utcnow().tz_convert("America/Chicago").tz_localize(None) - 
    pd.Timedelta('2D')
).tz_localize(None)

end_ts 

In [0]:
# lookback 7 days to cover holidays and long weekends when data doesn't get updated
# dc.collect_upsert_lmp(n_periods=7, daily_file=True)
range_df = dc.get_range_data_interval_daily_lmps(end_ts=end_ts,  n_periods=7)
range_df

In [0]:
parquet_files = [pf for pf in range_df if pf.endswith('.parquet')]
parquet_files

In [0]:
dc.upsert_lmp(parquet_files)